In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
ACCESS_KEY = "<YOUR_ACCESS_KEY>"
SECRET_KEY = "<YOUR_SECRET_KEY>"
MINIO_URL = "http://minio:9000"

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("HW2") \
    .config("spark.sql.adaptive.enabled", False) \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.sources.bucketing.enabled", True) \
    .config("spark.executor.memory", "450M") \
    .config("spark.driver.memory", "450M") \
    .config('spark.jars.packages', 
        "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-pom:1.12.365,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
    ) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_URL) \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-pom added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c1e405e5-032d-4d51-bee7-62d9653bc748;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-pom;1.12.365 in central
:: resolution report :: resolve 117ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	com.amazonaws#aws-java-sdk-pom;1.12.365 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|        

# Задание 1

## Входные данные 
- Файл с данными по оттоку телеком оператора в США (churn.csv)
- Справочник с названиями штатов (state.json)
- Справочник с численностью населения территорий (определяется полем area code) внутри штатов (state.json)
- Террия с численностью населения меньше 10_000 считается **мелкой**

## Что нужно сделать
1. Посчитать количество отточных и неотточных абонентов (поле churn), исключив **мелкие** территории
2. Отчет должен быть выполнен в разрезе **каждого штата** с его полным наименованием
3. Описать возникающие узкие места при выполнении данной операции
4. Применить один из способов оптимизации для ускорения выполнения запроса (при допущении, что справочник численности населения **сильно меньше** основных данных)
5. Если существует еще какой-то способ, применить также и его отдельно от п.4 (при допущении, что справочник численности населения **сопоставим по размеру** с основными данными)
6. Кратко описать реализованные способы и в чем их практическая польза

- P.S. Одним из выбранных способов должен быть Bucket specific join
- P.P.S. При обосновании предлагаем прикладывать запуска команды df.explain()

In [3]:
churn_df = spark.read.option("header", True).csv("s3a://input/data/churn.csv")
state_dict = spark.read.json("s3a://input/data/state.json")
pop_dict = spark.read.json("s3a://input/data/population.json")

24/06/20 09:37:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


### Решение

In [3]:
# Ваше решение в ячейках ниже ...

In [ ]:
# Здесь необходимо вывести результат:
# result.explain()
# result.show(truncate=False)

### Оптимизация 1

In [ ]:
# Ваше решение в ячейках ниже ...

In [ ]:
# Здесь необходимо вывести результат:
# result.explain()
# result.show(truncate=False)

### Оптимизация 2

In [ ]:
# Ваше решение в ячейках ниже ...

In [ ]:
# Здесь необходимо вывести результат:
# result.show(truncate=False)

# Задание 2

## Входные данные 

*skew_transactions.csv* - информация о длительности просомтра контента пользователям
колонки:
1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. watched_time — время просмотра в секундах

*catalogue.json* - каталог с описанием контента и метаинформации по нему
колонки:
1. type — тип элемента
2. duration — длительность в минутах (средняя длительность эпизода в случае с сериалами и многосерийными фильмами), округлённая до десятков
3. attributes — анонимизированные атрибуты данного элемента
4. availability — доступные права на элемент(subscription, purchase, rent)
5. feature_1 — анонимизированная вещественная переменная
6. feature_2 — анонимизированная вещественная переменная
7. feature_3 — анонимизированная порядковая переменная
8. feature_4 — анонимизированная вещественная переменная
9. feature_5 — анонимизированная вещественная переменная

## Что нужно сделать
1. Выполните join основных данных со справочником используя DataFrame API (по колонке id для контента - `element_uid`)
2. Описать проблему в датасетах с точки зрения обработки Spark
3. Решить задачу любым способом
4. Решить задачу с помощью salt-join подхода

P.S. Как вы можете заметить при просмотре данных по пользователями, нужный нам ключ для операции будет перекошен (90% строк представлены на фильм, очень популярный среди смотревших) - это нужно доказать в рамках п.2

### Решение 

In [ ]:
# Ваше решение в ячейках ниже ...

In [ ]:
# Здесь необходимо вывести результат:
# result.explain()
# result.show(truncate=False)

### Решение с оптимизацией

In [ ]:
# Ваше решение в ячейках ниже ...

In [ ]:
# Здесь необходимо вывести результат:
# result.explain()
# result.show(truncate=False)

# Задание 3

## Входные данные 

*cut_transactions.csv*  — информация о длительности просомтра контента пользователям

Описание фичей в cut_transactions.csv: 
1. user_uid — уникальный идентификатор пользователя
2.  element_uid — уникальный идентификатор контента
3.  watched_time — время просмотра в секундах

*cut_ratings.csv*  — информация об оценках, поставленных пользователями

Описание фичей в cut_ratings.csv: 
1. user_uid — уникальный идентификатор пользователя 
2. element_uid — уникальный идентификатор контента 
3. rating — поставленный пользователем рейтинг

*ids.csv*  — выборка пользователей
Описание фичей в ids.csv: 
1. user_uid — уникальный идентификатор пользователя 


## Что нужно сделать
Для каждого пользователя из выборки посчитать:
1. Максимальное и минимальное время просмотра фильмов с оценками 8, 9 и 10 
2. Название фичи должно быть в формате feat_агрегирующая_функция_watched_time_rating_оценка. 
3. Если у пользователь не ставил оценки 8, 9 и 10 то значение фичей должно быть null
4. Описать принятые при разработки кода решения и возможные оптимизации

P.S. На каждом этапе обработки должно быть должны агрегироваться минимально возможные объемы данных (сокращаем затраты на shuflle)

### Решение

In [ ]:
# Ваше решение в ячейках ниже ...

In [ ]:
# Здесь необходимо вывести результат:
# result.explain()
# result.show(truncate=False)